#### What are joins?

A join brings together two sets of data, the left and the right by comparing the value of one or more keys from the left and right and evaluating the result that determines whether Spark should bring the left and right sets of data.

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

#### Start a SparkSession where we can try out different dataframe joins.

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T

spark = SparkSession.builder.appName("DataFrame Joins - UFC data").getOrCreate()

<IPython.core.display.Javascript object>

In [3]:
spark

<IPython.core.display.Javascript object>

#### Load data

In [4]:
import os
from os.path import isfile, join

loc = os.path.abspath("")
data_loc = f"{loc}/data/ufcdata/"
data_files = [f for f in os.listdir(data_loc) if isfile(join(data_loc, f))]
print(data_files)

['raw_total_fight_data.csv', 'raw_fighter_details.csv', 'data.csv']


<IPython.core.display.Javascript object>

Load all the files in a dictionary and set the keys as the file names (for easy access).

In [5]:
data = {}
for file in data_files:
    data[file[:-4]] = spark.read.csv(f"{data_loc}{file}", inferSchema=True, header=True)
    data[file[:-4]].createOrReplaceTempView(f"data_{file[:-4]}")
print(data.keys())

dict_keys(['raw_total_fight_data', 'raw_fighter_details', 'data'])


<IPython.core.display.Javascript object>

We want to use only the data from the 2017 onwards and we'll create two new columns, with the Winner name and the Loser name rather than have Red and Blue as we have currently.

In [6]:
data["data"] = (
    data["data"]
    .where(F.col("date") > "2017-01-01")
    .withColumn(
        "Winner_name",
        F.when(F.col("Winner") == "Red", F.col("R_fighter")).otherwise(
            F.col("B_fighter")
        ),
    )
    .withColumn(
        "Loser_name",
        F.when(F.col("Winner") == "Red", F.col("B_fighter")).otherwise(
            F.col("R_fighter")
        ),
    )
)

<IPython.core.display.Javascript object>

Now we can group by the winner and the loser name, we create two dataframes where we count the number of wins and losses for each fighter.

I renamed the columns Winner_name and Loser_name to a common name so we can join the two in the next step. 


In [7]:
losses = (
    data["data"]
    .withColumnRenamed("Loser_name", "name")
    .groupBy("name")
    .count()
    .select("name", F.col("count").alias("losses"))
)

wins = (
    data["data"]
    .withColumnRenamed("Winner_name", "name")
    .groupBy("name")
    .count()
    .select("name", F.col("count").alias("wins"))
)

<IPython.core.display.Javascript object>

Before we start the join we need to count the total distinct number of figthers that fought in any of the red or blue corners.

This number servers as a check for when we do our join, we need to make sure that they match. 

The reason is that we need to make sure we include all the fighters regardless whether they won or lost any fights.

In [8]:
red = data["data"].select("R_fighter").distinct()
blue = data["data"].select("B_fighter").distinct()
check = red.union(blue).distinct().count()
print(check)

763


<IPython.core.display.Javascript object>

And here are the distinct counts for each of the wins and losses dataframes.

In [9]:
print(wins.distinct().count())
print(losses.distinct().count())

535
645


<IPython.core.display.Javascript object>

#### Join types


1. **Inner joins** (default) - keep the rows that exist in both left and right dataframes

In this scenario, we keep only the fighters that have at least a win and a loss. If the fighter doesn't have any wins or losses he won't be included in the new dataframe.

Inner joins are the default behaviour, the intersection, and they aren't useful in our case as we want to include fighters regardless whether they had losses or wins or not. 

In [10]:
win_loss_data = wins.join(losses, "name")
print(win_loss_data.distinct().count())

417


<IPython.core.display.Javascript object>

2. **Left joins** - keep the rows with keys in the left dataframe

In this scenario we keep only the fighters that had at least a win. 

Again, it's not useful to us as we ignore the figthers that only lost, 215 fighters in this case.

In [11]:
win_loss_data = wins.join(losses, "name", "left")
print(win_loss_data.distinct().count())

535


<IPython.core.display.Javascript object>

3. **Right joins** - keep the rows with keys in the right dataframe

In this scenario we keep only the fighters that had at least a loss. 


In [12]:
win_loss_data = wins.join(losses, "name", "right")
print(win_loss_data.distinct().count())

645


<IPython.core.display.Javascript object>

4. **Outer joins** - keep rows with keys in either the left or right dataframe

This is the behaviour we want in this case, to make sure we include all fighters.

In [13]:
win_loss_data = wins.join(losses, "name", "outer")
print(win_loss_data.distinct().count())

763


<IPython.core.display.Javascript object>

In [14]:
win_loss_data.orderBy(F.desc("wins")).show(5)

+--------------------+----+------+
|                name|wins|losses|
+--------------------+----+------+
|       Thiago Santos|   8|     1|
|     Israel Adesanya|   6|  null|
|Alexander Volkano...|   6|  null|
|Anthony Rocco Martin|   6|     1|
|   Aljamain Sterling|   6|     2|
+--------------------+----+------+
only showing top 5 rows



<IPython.core.display.Javascript object>

Now let's join this newly created dataframe with the raw_fighter_details dataframe. 

We're going to use a left join as we only want to keep the keys from the left dataframe.

In [15]:
win_loss_data = win_loss_data.join(
    data["raw_fighter_details"].withColumnRenamed("fighter_name", "name"),
    "name",
    "left",
)

<IPython.core.display.Javascript object>

Let's calculate each fighter's age and just select the name, wins or losses and age. 

In [16]:
win_loss_data = (
    win_loss_data.withColumn("birthday", F.to_date(F.col("DOB"), "MMM dd, yyyy"))
    .withColumn(
        "age", F.round(F.months_between(F.current_date(), F.col("birthday")) / 12, 1)
    )
    .select("name", "wins", "losses", "age")
)

<IPython.core.display.Javascript object>

Order by age and the number of wins to be higher than an arbitrary 5.

In [17]:
win_loss_data.where(F.col("losses").isNull()).orderBy(
    F.asc("age"), F.desc("wins")
).where("wins > 5").show()

+--------------------+----+------+----+
|                name|wins|losses| age|
+--------------------+----+------+----+
|     Israel Adesanya|   6|  null|30.5|
|      Dustin Poirier|   6|  null|31.0|
|Alexander Volkano...|   6|  null|31.3|
|        Kamaru Usman|   6|  null|32.7|
+--------------------+----+------+----+



<IPython.core.display.Javascript object>